# Lecture 3: Pandas [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) 4

* [`pandas.merge(...)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)

## Imports

In [1]:
import pandas as pd

## [`pandas.merge(...)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)

Let's begin by re-creating the `DataFrame`s from the slides. To do so, we now use a dictionary, whose keys are the column names and whose values are the column content.

In [2]:
student_programmes = pd.DataFrame({'student': ['Bob', 'Jake', 'Lisa', 'Sue'],
                                   'programme': ['BIM', 'MiM', 'IM', 'SCM']})
student_programmes

,student,programme
0,Bob,BIM
1,Jake,MiM
2,Lisa,IM
3,Sue,SCM


In [3]:
student_enrolments = pd.DataFrame({'student': ['Lisa', 'Bob', 'Jake', 'Liz'],
                                   'enrolment': [2004, 2008, 2012, 2014]})
student_enrolments

,student,enrolment
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Liz,2014


In [4]:
programme_directors = pd.DataFrame({'programme': ['BIM', 'MiM', 'IM'],
                                    'Director': ['Carly', 'Guido', 'Steve']})
programme_directors

,programme,Director
0,BIM,Carly
1,MiM,Guido
2,IM,Steve


### Default: Natural Inner Join

A natural join aligns all columns of the same name.

In [5]:
pd.merge(student_programmes, student_enrolments)

,student,programme,enrolment
0,Bob,BIM,2008
1,Jake,MiM,2012
2,Lisa,IM,2004


This is equivalent to:

In [6]:
pd.merge(student_programmes, student_enrolments,
         on='student', how='inner')

,student,programme,enrolment
0,Bob,BIM,2008
1,Jake,MiM,2012
2,Lisa,IM,2004


### Left Join

We keep everything in `student_programmes` and the matching observations from `student_enrolments`.

In [7]:
pd.merge(student_programmes, student_enrolments,
         how='left')

,student,programme,enrolment
0,Bob,BIM,2008.0
1,Jake,MiM,2012.0
2,Lisa,IM,2004.0
3,Sue,SCM,NaN


Whenever we leave out `on` it defaults to a natural join, e.g. a natural left join.

### Right Join
We keep everything in `student_enrolments` and the matching observations from `student_programmes`.

In [8]:
pd.merge(student_programmes, student_enrolments,
         how='right')

,student,programme,enrolment
0,Bob,BIM,2008
1,Jake,MiM,2012
2,Lisa,IM,2004
3,Liz,NaN,2014


### Outer Join

We keep all observations.

In [9]:
pd.merge(student_programmes, student_enrolments,
         how='outer')

,student,programme,enrolment
0,Bob,BIM,2008.0
1,Jake,MiM,2012.0
2,Lisa,IM,2004.0
3,Sue,SCM,NaN
4,Liz,NaN,2014.0


## Joining on Columns with Different Names

So far, we could always use a natural join, because the columns we wanted to join on had the same name in different `DataFrame`s. If that's not the case, or if we want to join only on some of the same-named columns, we can specify which columns we want to join on using the `on`, `left_on`, and `right_on` arguments.

We've already seen `on`, which lets us specify the join-column(s) in both `DataFrame`s. This will always be a subset of the natural join columns.

To specify the join columns separately for the left and right `DataFrame`, we can use the appropriately named `left_on` and `right_on`.

For example, let's say we have another `DataFrame` that contains the dean for each year:

In [10]:
deans = pd.DataFrame({'dean': ['Steve', 'Steve', 'Ansgar', 'Ansgar'],
                      'year': [2004, 2008, 2012, 2020]})
deans

,dean,year
0,Steve,2004
1,Steve,2008
2,Ansgar,2012
3,Ansgar,2020


If we want to link each student to the dean in office in the year of her enrolment, a natural join won't do anymore:

In [11]:
pd.merge(student_enrolments, deans)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

Instead, we have to specify the join columns:

In [12]:
pd.merge(student_enrolments, deans,
         left_on='enrolment', right_on='year',
         how='left')

,student,enrolment,dean,year
0,Lisa,2004,Steve,2004.0
1,Bob,2008,Steve,2008.0
2,Jake,2012,Ansgar,2012.0
3,Liz,2014,NaN,NaN


### Joining on Indices

We have to employ similar tactics if we want to join on indices. For example, let's say we have data on the skills that each programme imparts in a separate `DataFrame`:

In [13]:
programme_skills = pd.DataFrame({'programme': ['BIM', 'BIM', 'MiM', 'IM'],
                                 'skills': ['Python', 'Java', 'Talking', None]})
programme_skills.set_index('programme', inplace=True)
programme_skills

,skills
programme,
BIM,Python
BIM,Java
MiM,Talking
IM,None


Now, `programme` is not a column but an explicit index. To join `programme_skills` with `student_programmes` we need to use one of the `left_index`/`right_index` arguments. They take a `bool` value that indicates whether to join on the left or right `DataFrame`'s index. In this case, we're joining on the right `DataFrame`'s index:

In [14]:
pd.merge(student_programmes, programme_skills,
         left_on='programme', right_index=True,
         how='inner')

,student,programme,skills
0,Bob,BIM,Python
0,Bob,BIM,Java
1,Jake,MiM,Talking
2,Lisa,IM,None


We also need to specify which column to join on in the left `DataFrame`, because there is no overlapping set of columns and indices.

© 2023 Philipp Cornelius